In [10]:
#pip install bayesian-optimization


Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from bayes_opt import BayesianOptimization
#nltk.download('punkt')
#nltk.download('stopwords')

In [69]:
# Load the dataset
data = pd.read_csv(r'C:\Users\ayush\Downloads\0000000000002747_training_twitter_x_y_train.csv')

# Select relevant columns
data = data[['text', 'airline_sentiment']]
data['airline_sentiment'] = data['airline_sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1})

# Text preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lower case
    tokens = [word for word in tokens if word.isalnum()]  # Remove punctuation and numbers
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)

# Apply text preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Convert text data to numerical data using TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['text']).toarray()
y = data['airline_sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Define the optimization function for Random Forest
def optimize_rf(n_estimators, max_depth, min_samples_split):
    # Convert float parameters to integers since these are required by RandomForestClassifier
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    
    # Create the Random Forest model with given parameters
    rf_clf = RandomForestClassifier(n_estimators=n_estimators, 
                                    max_depth=max_depth, 
                                    min_samples_split=min_samples_split, 
                                    random_state=42)
    
    # Perform cross-validation to evaluate the model's performance
    scores = cross_val_score(rf_clf, X_train, y_train, cv=3, scoring='accuracy')
    
    
    # Return the mean of the cross-validation scores as the objective function value
    return scores.mean()
    


In [71]:
from bayes_opt import BayesianOptimization

# Define the parameter bounds for the Bayesian optimizer
param_bounds = {
    'n_estimators': (10, 300),        # Number of trees in the forest
    'max_depth': (3, 30),             # Maximum depth of the tree
    'min_samples_split': (2, 20)      # Minimum number of samples required to split an internal node
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(f=optimize_rf, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=20)

# Extract the best parameters found by the optimizer
best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_samples_split'] = int(best_params['min_samples_split'])

# Output the best parameters
print("Best Parameters:", best_params)


|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.6292    | 13.11     | 19.11     | 222.3     |
| 2         | 0.6447    | 19.16     | 4.808     | 55.24     |
| 3         | 0.6256    | 4.568     | 17.59     | 184.3     |
| 4         | 0.6473    | 22.12     | 2.371     | 291.3     |
| 5         | 0.6606    | 25.48     | 5.822     | 62.73     |
| 6         | 0.6722    | 28.8      | 2.515     | 67.92     |
| 7         | 0.6792    | 30.0      | 5.573     | 85.4      |
| 8         | 0.6753    | 29.88     | 5.917     | 86.38     |
| 9         | 0.675     | 30.0      | 12.46     | 76.41     |
| 10        | 0.6454    | 20.68     | 2.0       | 79.33     |
| 11        | 0.6793    | 30.0      | 4.038     | 77.21     |
| 12        | 0.6256    | 3.0       | 20.0      | 10.0      |
| 13        | 0.6775    | 30.0      | 2.0       | 140.3     |
| 14        | 0.6475    | 22.45     | 12.11     | 137.0     |
| 15    

In [81]:
# Initialize the model with the best parameters
rf_clf = RandomForestClassifier(n_estimators=best_params['n_estimators'], 
                                max_depth=best_params['max_depth'], 
                                min_samples_split=best_params['min_samples_split'], 
                                random_state=42)

# Train the model
rf_clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=30, min_samples_split=4, n_estimators=77,
                       random_state=42)

In [93]:
# Initialize the Naive Bayes classifier
nb_clf = MultinomialNB()
nb_clf.fit(X_train,y_train)

# Combine the classifiers into a voting classifier
voting_clf = VotingClassifier(estimators=[('rf', rf_clf), ('nb', nb_clf)], voting='soft')

# Train the ensemble model
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('nb', MultinomialNB())],
                 voting='soft')

In [95]:
# Make predictions
y_pred = voting_clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print a detailed classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.7523
              precision    recall  f1-score   support

          -1       0.74      0.98      0.85      1356
           0       0.72      0.32      0.44       458
           1       0.87      0.46      0.60       382

    accuracy                           0.75      2196
   macro avg       0.78      0.59      0.63      2196
weighted avg       0.76      0.75      0.72      2196



In [87]:
# Initialize the classifiers
rf_clf = RandomForestClassifier(random_state=42)
nb_clf = MultinomialNB()

# Create the VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('rf', rf_clf),
    ('nb', nb_clf)
], voting='soft')


In [76]:
# Apply GridSearchCV
grid_search = GridSearchCV(estimator=voting_clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)


NameError: name 'GridSearchCV' is not defined

In [ ]:
# Best parameters found by GridSearchCV
print("Best parameters found: ", grid_search.best_params_)

# Best estimator
best_voting_clf = grid_search.best_estimator_

# Make predictions with the best estimator
y_pred = best_voting_clf.predict(X_test)

# Evaluate the best model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import joblib

# Save the best model to a file
joblib.dump(best_voting_clf, 'best_voting_classifier_twitter_sentiment.pkl')


In [ ]:
# Load the model from file
loaded_model = joblib.load('best_voting_classifier_twitter_sentiment.pkl')

# Make predictions using the loaded model
y_pred_loaded = loaded_model.predict(X_test)
